In [868]:
import numpy as np
import pandas as pd
import sqlite3

In [869]:
df = None
df2 = None

In [870]:
table = 'apartments'

In [871]:
all_columns = ['apt_id', 'name', 'address', 'unit', 'sqft', 'bed', 'bath', 'price', 'city', 'state', 'zip_code', 'description', 'details', 'url', 'date']

In [872]:
columns = ['apt_id', 'name', 'address', 'sqft', 'bed', 'bath', 'price', 'zip_code']

In [873]:
conn = sqlite3.connect('housing.db')
cur = conn.cursor()

statement = 'SELECT ' + ', '.join(all_columns) + f' FROM {table}'

cur.execute(statement)

data = cur.fetchall()

conn.commit()

conn.close()

In [874]:
if df is None:
    df = pd.DataFrame(data, columns=all_columns)


else:
    df2 = pd.DataFrame(data, columns=all_columns)

    df = df.append(df2)


In [875]:
df.replace(' ', '', inplace=True) # whitespace to blank
df.replace('', np.nan, inplace=True) # blank to NaN
df.dropna(inplace=True) # drop NaN rows

df.sqft = df.sqft.str.replace('sqft','').str.replace(',','').str.strip()

df.price = df.price.str.replace('$','').str.replace(',','').str.strip()

df.bath = df.bath.str.replace('ba','').str.strip()

df.bed = df.bed.str.replace('bd','').str.lower().replace('studio',0).str.strip()

df.bed = df.bed.replace(np.nan, 0)

df = df[~df.price.str.contains('-', na=False)]

df = df.astype({'sqft': 'int32', 'price': 'int32', 'bath': 'float32', 'bed': 'float32'})

In [876]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11513856 entries, 0 to 11525051
Data columns (total 15 columns):
apt_id         int64
name           object
address        object
unit           object
sqft           int32
bed            float32
bath           float32
price          int32
city           object
state          object
zip_code       int64
description    object
details        object
url            object
date           object
dtypes: float32(2), int32(2), int64(2), object(9)
memory usage: 1.2+ GB


In [877]:
df.to_csv('rentals.csv')

In [878]:
df.drop(columns='apt_id').to_sql(name='apartments', con=sqlite3.connect('housing.db'), if_exists='append', index=False)